In [3]:
# Imports 
from lhotse import CutSet, RecordingSet, align_with_torchaudio, annotate_with_whisper
from tqdm import tqdm
from pprint import pprint
from dataclasses import asdict
import torch


/home/kastan/utils/miniconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
cut_extract

{'id': 'audio_testrick_roll',
 'start': 0.0,
 'duration': 212.044625,
 'channel': 0,
 'supervisions': [{'id': 'audio_testrick_roll-000000',
   'recording_id': 'audio_testrick_roll',
   'start': 0.0,
   'duration': 28.72,
   'channel': 0,
   'text': '[♪ OUTRO CONT %.',
   'language': 'en',
   'speaker': None,
   'gender': None,
   'custom': None,
   'alignment': {'word': [AlignmentItem(symbol='OUTRO', start=15.49075, duration=3.8626875, score=0.9572148400291298),
     AlignmentItem(symbol='CONT', start=19.4135, duration=1.16075, score=0.8830821210767368)]}},
  {'id': 'audio_testrick_roll-000001',
   'recording_id': 'audio_testrick_roll',
   'start': 28.72,
   'duration': 6.0,
   'channel': 0,
   'text': "You won't get this wrong any other guy",
   'language': 'en',
   'speaker': None,
   'gender': None,
   'custom': None,
   'alignment': {'word': [AlignmentItem(symbol='YOU', start=29.4624375, duration=1.0251875, score=0.8589585355903003),
     AlignmentItem(symbol="WON'T", start=30.6263

In [28]:
# Group every 15 captions
def group_n_captions(cut, n = 15):
    all_dict_lists = []
    curr_dict_list = []
    curr_string = ""
    for supervision in cut['supervisions']:
        for word in supervision['alignment']['word']:
            curr_string += word.symbol + " "
            new_dict = {"word": word.symbol, "start": word.start, "end": word.start + word.duration}
            curr_dict_list.append(new_dict)
            if len(curr_dict_list) == n:
                all_dict_lists.append({"overall_caption": curr_string[0: -1], "n_words_list": curr_dict_list})
                curr_dict_list = []
                curr_string = ""
    return all_dict_lists

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

recordings = RecordingSet.from_dir("/home/kastan/thesis/whisper_audio/audio_test", pattern="*.wav")
cuts = annotate_with_whisper(recordings, device = device)
cuts_aligned = align_with_torchaudio(cuts)

Scanning audio files (*.wav): 1it [00:00, 954.12it/s]


In [21]:
for cut in cuts_aligned:
    cut_extract = asdict(cut)
    

In [30]:
group_rickroll = group_n_captions(cut_extract)


In [91]:
for ind_dict in group_rickroll:
    print(len(ind_dict["overall_caption"].split()))

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15


In [88]:
from transformers import GPT2Tokenizer, OPTModel
import tensorflow as tf

tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-350m")
model = OPTModel.from_pretrained("facebook/opt-350m")

embedding_list = []
for ind_dict in group_rickroll:
    print(ind_dict["overall_caption"])
    inputs = tokenizer(ind_dict["overall_caption"], return_tensors="pt")
    print((inputs['input_ids'][0]))
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    embedding_list.append(last_hidden_states)


OUTRO CONT YOU WON'T GET THIS WRONG ANY OTHER GUY I JUST WANNA TELL YOU
tensor([    2,  5061,  6997,   673, 19137, 10540,   305,  2191,   108,   565,
         7777, 10652,  8692,  9672, 16005, 16536, 22294,   975,    38, 34157,
          305,  1889,  4444, 13180,  6006, 10540])
HOW I FEEL YOU GOT TO MAKE YOU UNDERSTAND NEVER GONNA GIVE YOU UP NEVER
tensor([    2, 44897,    38, 16852,  3721, 10540, 42007,  3842, 39853, 10540,
        34290,  4014,  5945, 37603,   272,  2191,  4444,   272, 10002, 10540,
         5570, 37603])
GONNA LET YOU DOWN NEVER GONNA RUN AROUND IT HE'S HURT YOU NEVER GONNA MAKE
tensor([    2,   534,  2191,  4444, 39888, 10540, 20685, 37603,   272,  2191,
         4444, 35557,  5495, 18103,  3779, 12925,   108,   104,   289,  2492,
          565, 10540, 37603,   272,  2191,  4444, 39853])
YOU CRY NEVER GONNA SAY GOODBYE NEVER GONNA TELL THE LIE AND HURT YOU WE'VE
tensor([    2, 38158,  4307,   975, 37603,   272,  2191,  4444, 29391, 31708,
        19058,   717, 3760

In [81]:
embedding_list[0].shape

torch.Size([1, 26, 512])

In [61]:
for i in range(22):
    for j in range(512):
        if embedding_list[0][0][i][j] != embedding_list[1][0][i][j]:
            print(embedding_list[0][0][j], embedding_list[1][0][j])

tensor([-1.3540e+00, -2.5788e-01, -3.0939e-01,  3.2718e-01, -9.8934e-01,
        -2.3015e+00, -7.3821e-01, -9.9575e-02, -1.0576e+00, -1.5872e+00,
         3.6742e-01,  9.2745e-01, -7.2668e-01, -2.0221e-01, -7.6863e-01,
        -1.1988e+00, -8.4748e-01, -3.3735e-01,  5.1817e-01,  2.6762e-01,
        -9.2919e+00, -6.6914e-02, -1.1788e+00,  2.6636e-02, -5.2492e-01,
         1.5562e-01, -5.2755e-02, -3.1586e+00,  7.0485e-01,  5.3312e-02,
        -7.5138e-01, -1.4547e+00, -9.1142e-01, -4.3719e-02, -4.3541e+00,
        -9.1938e-01,  9.2125e-01, -4.0929e+00, -2.0001e+00, -3.1855e-01,
        -2.9570e-01,  6.8197e-01, -5.0765e-01, -6.4116e+00,  2.7609e-01,
         2.4526e-01,  7.0180e-01,  1.1647e+00,  3.8459e-01,  7.7488e-01,
        -4.1853e-01, -2.5265e+00, -1.3062e+00,  3.2322e-01,  1.4377e+00,
        -8.7600e-01,  1.9577e+00, -4.2378e-01, -2.4190e-02,  8.8814e-02,
         1.2626e+00,  7.7488e-01,  3.8269e-01,  8.5246e-01, -2.4775e-01,
        -1.0872e+00,  6.9244e-01, -2.3151e+01, -8.3

IndexError: index 22 is out of bounds for dimension 0 with size 22